In [1]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install audiocraft # install the missing module

In [3]:
import os
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
import ipywidgets as widgets
from IPython.display import display

# **Music Generation**

In [ ]:
# Initialize MusicGen model
model = MusicGen.get_pretrained('facebook/musicgen-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Predefined options for genre, style, instrument, tempo, and mood
genres = ["Experimental", "Classical", "Ambient", "Jazz", "Rock"]
styles = ["Noise", "Melodic", "Rhythmic", "Atmospheric", "Harmonic"]
instruments = ["Modular Synth", "Piano", "Guitar", "Drums", "Violin"]
tempos = ["Slow", "Medium", "Fast"]
moods = ["Happy", "Sad", "Mysterious", "Energetic"]

# Dropdown widgets for genre, style, instrument, tempo, and mood selection
genre_dropdown1 = widgets.Dropdown(options=genres, description='Genre 1:')
style_dropdown1 = widgets.Dropdown(options=styles, description='Style 1:')
instrument_dropdown1 = widgets.Dropdown(options=instruments, description='Instrument 1:')
tempo_dropdown1 = widgets.Dropdown(options=tempos, description='Tempo 1:')
mood_dropdown1 = widgets.Dropdown(options=moods, description='Mood 1:')

genre_dropdown2 = widgets.Dropdown(options=genres, description='Genre 2:')
style_dropdown2 = widgets.Dropdown(options=styles, description='Style 2:')
instrument_dropdown2 = widgets.Dropdown(options=instruments, description='Instrument 2:')
tempo_dropdown2 = widgets.Dropdown(options=tempos, description='Tempo 2:')
mood_dropdown2 = widgets.Dropdown(options=moods, description='Mood 2:')

# Dropdown widget for description file selection
back_directory = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/Selected5Images'
input_directory = '/content/drive/MyDrive/Hackathons/NASASpaceChallenge/Descriptions'
json_files = [f for f in os.listdir(input_directory) if f.endswith('.json')]
description_dropdown = widgets.Dropdown(options=json_files, description='Description File:')
confirm_button = widgets.Button(description="Generate Music")

# Function to find an image with any common extension
def find_image_with_name(directory, base_name):
    """Find an image file with a given base name in the specified directory."""
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)) and filename.startswith(base_name):
            return os.path.join(directory, filename)
    return None

# Function to handle the confirm button click
def on_button_click(b):
    # Fetch selected options for both parts
    selected_genre1 = genre_dropdown1.value
    selected_style1 = style_dropdown1.value
    selected_instrument1 = instrument_dropdown1.value
    selected_tempo1 = tempo_dropdown1.value
    selected_mood1 = mood_dropdown1.value

    selected_genre2 = genre_dropdown2.value
    selected_style2 = style_dropdown2.value
    selected_instrument2 = instrument_dropdown2.value
    selected_tempo2 = tempo_dropdown2.value
    selected_mood2 = mood_dropdown2.value

    selected_file = description_dropdown.value
    selected_file_path = os.path.join(input_directory, selected_file)

    # Load the description
    try:
        with open(selected_file_path, 'r') as f:
            description_data = json.load(f)
    except Exception as e:
        print(f"Error: Failed to load the description file '{selected_file_path}'. Reason: {e}")
        return

    # Access the correct key dynamically (image name)
    image_name = list(description_data.keys())[0]
    original_description = description_data[image_name]

    # Create prompts for both parts of the music
    prompt_part1 = f"{original_description} Set to the {selected_genre1} genre, with a {selected_style1} style, featuring {selected_instrument1}, at a {selected_tempo1} tempo, and a {selected_mood1} mood."
    prompt_part2 = f"{original_description} Set to the {selected_genre2} genre, with a {selected_style2} style, featuring {selected_instrument2}, at a {selected_tempo2} tempo, and a {selected_mood2} mood."

    try:
        # Generate the first 15 seconds of music
        print(f"Generating first 15 seconds... for {image_name}")
        model.set_generation_params(duration=15)
        wav_part1 = model.generate([prompt_part1])

        # Generate the next 15 seconds of music
        print(f"Generating next 15 seconds... for {image_name}")
        model.set_generation_params(duration=15)
        wav_part2 = model.generate([prompt_part2])

        # Concatenate both parts into one audio file
        combined_wav = torch.cat((wav_part1[0], wav_part2[0]), dim=-1)

        # Create a filename with the selections from both parts
        audio_filename_prefix = f"{image_name}_{selected_genre1}_{selected_style1}_{selected_instrument1}_to_{selected_genre2}_{selected_style2}_{selected_instrument2}"
        audio_filename = os.path.join(input_directory, f'{audio_filename_prefix}_combined_music.wav')

        # Save the concatenated audio in the same directory
        torchaudio.save(audio_filename, combined_wav.cpu(), sample_rate=model.sample_rate)

        # Display the combined audio for playback
        display(Audio(audio_filename, autoplay=False))

        print(f"Music generated and saved as {audio_filename}")

    except Exception as e:
        print(f"An error occurred while generating music: {e}")
        return

    # Visualize the waveform with the generated audio and the selected background image
    background_image_path = find_image_with_name(back_directory, image_name)

    if not os.path.exists(background_image_path):
        print(f"Error: Background image '{background_image_path}' not found.")
        return

    # Load the background image
    background = cv2.imread(background_image_path)
    if background is None:
        print(f"Error: Failed to load background image from '{background_image_path}'. The file might be corrupted or not a valid image format.")
        return

    # Resize the image to match video resolution
    background = cv2.resize(background, (1280, 720))

    # Continue with your visualization code
    output_file = os.path.join(input_directory, f"{audio_filename_prefix}_waveform_video.mp4")
    fps = 30  # Define the FPS for the video
    visualize_waveform(audio_filename, background_image_path, output_file, fps)

# Confirm button click event
confirm_button.on_click(on_button_click)

# Display dropdowns and confirm button
display(genre_dropdown1, style_dropdown1, instrument_dropdown1, tempo_dropdown1, mood_dropdown1,
        genre_dropdown2, style_dropdown2, instrument_dropdown2, tempo_dropdown2, mood_dropdown2,
        description_dropdown, confirm_button)

# Function to generate frames for the waveform visualization
def generate_frame(t, y, sr, background, fps):
    """Generate a video frame with the audio waveform at a specific time."""
    frame = background.copy()

    frame_width = frame.shape[1]
    samples_per_frame = int(sr / fps)
    start_sample = int(t * sr)
    end_sample = start_sample + samples_per_frame

    # Extract the portion of the audio corresponding to this frame
    if end_sample < len(y):
        audio_segment = y[start_sample:end_sample]
    else:
        audio_segment = y[start_sample:]

    if len(audio_segment) > 0:
        audio_amplitude = np.abs(audio_segment).mean()
        audio_segment = audio_segment / np.max(np.abs(y))
    else:
        audio_amplitude = 0

    waveform_height = int(frame.shape[0] / 2)
    y_values = (audio_segment * waveform_height).astype(int)

    # Adjust brightness, contrast, and saturation based on the audio amplitude
    brightness_factor = 1 + audio_amplitude  # Brightness changes with amplitude
    contrast_factor = 1 + (audio_amplitude / 2)  # Slight contrast change
    saturation_factor = 1 + (audio_amplitude / 3)  # Slight saturation change

    frame = adjust_brightness(frame, brightness_factor)
    frame = adjust_contrast(frame, contrast_factor)
    frame = adjust_saturation(frame, saturation_factor)

    # Draw the waveform on the frame, with color based on amplitude
    for x in range(len(y_values) - 1):
        x1 = int(frame_width * (x / len(y_values)))
        y1 = frame.shape[0] // 2 - y_values[x]
        x2 = int(frame_width * ((x + 1) / len(y_values)))
        y2 = frame.shape[0] // 2 - y_values[x + 1]

        # Change waveform color based on amplitude
        color_intensity = int(255 * audio_amplitude)
        color = (0, color_intensity, 255 - color_intensity)  # Dynamic color change
        cv2.line(frame, (x1, y1), (x2, y2), color, 1)

    return frame

# Functions to adjust image properties
def adjust_brightness(image, factor):
    """Adjust the brightness of an image by a given factor."""
    return np.clip(image * factor, 0, 255).astype(np.uint8)

def adjust_contrast(image, factor):
    """Adjust the contrast of an image by a given factor."""
    mean = np.mean(image)
    return np.clip((image - mean) * factor + mean, 0, 255).astype(np.uint8)

def adjust_saturation(image, factor):
    """Adjust the saturation of an image by a given factor."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  # Convert to HSV color space
    hsv_image[..., 1] = np.clip(hsv_image[..., 1] * factor, 0, 255)  # Adjust saturation
    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)  # Convert back to BGR

# Visualization function
def visualize_waveform(audio_file, background_image_path, output_file='output.mp4', fps=30):
    """Visualizes the audio waveform with a background image and generates a video."""
    y, sr = librosa.load(audio_file, sr=None)
    background = cv2.imread(background_image_path)

    if background is None:
        print(f"Error: Failed to load background image from '{background_image_path}'. Exiting visualization.")
        return

    background = cv2.resize(background, (1280, 720))

    video_clip = VideoClip(lambda t: generate_frame(t, y, sr, background, fps), duration=librosa.get_duration(y=y, sr=sr))
    audio_clip = AudioFileClip(audio_file)

    video_clip = video_clip.set_audio(audio_clip)
    video_clip.write_videofile(output_file, fps=fps)

    print(f"Waveform visualization saved to {output_file}.")


Dropdown(description='Genre 1:', options=('Experimental', 'Classical', 'Ambient', 'Jazz', 'Rock'), value='Expe…

Dropdown(description='Style 1:', options=('Noise', 'Melodic', 'Rhythmic', 'Atmospheric', 'Harmonic'), value='N…

Dropdown(description='Instrument 1:', options=('Modular Synth', 'Piano', 'Guitar', 'Drums', 'Violin'), value='…

Dropdown(description='Tempo 1:', options=('Slow', 'Medium', 'Fast'), value='Slow')

Dropdown(description='Mood 1:', options=('Happy', 'Sad', 'Mysterious', 'Energetic'), value='Happy')

...

Button(description='Generate Music', style=ButtonStyle())

Generating first 15 seconds... for SolarFlareStorm
Generating next 15 seconds... for SolarFlareStorm


Music generated and saved as /content/drive/MyDrive/Hackathons/NASASpaceChallenge/Descriptions/SolarFlareStorm_Ambient_Noise_Modular Synth_to_Experimental_Melodic_Modular Synth_combined_music.wav
Moviepy - Building video /content/drive/MyDrive/Hackathons/NASASpaceChallenge/Descriptions/SolarFlareStorm_Ambient_Noise_Modular Synth_to_Experimental_Melodic_Modular Synth_waveform_video.mp4.
MoviePy - Writing audio in SolarFlareStorm_Ambient_Noise_Modular Synth_to_Experimental_Melodic_Modular Synth_waveform_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Hackathons/NASASpaceChallenge/Descriptions/SolarFlareStorm_Ambient_Noise_Modular Synth_to_Experimental_Melodic_Modular Synth_waveform_video.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Hackathons/NASASpaceChallenge/Descriptions/SolarFlareStorm_Ambient_Noise_Modular Synth_to_Experimental_Melodic_Modular Synth_waveform_video.mp4
Waveform visualization saved to /content/drive/MyDrive/Hackathons/NASASpaceChallenge/Descriptions/SolarFlareStorm_Ambient_Noise_Modular Synth_to_Experimental_Melodic_Modular Synth_waveform_video.mp4.
